In [ ]:
import magmaforge
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def _convert_temp_index(df, T_unit):
    if T_unit == 'C':
        df.index = df.index - 273.15
        T_label = 'Temp [C]'
    elif T_unit == 'K':
        T_label = 'Temp [K]'
    else:
        assert False, 'Not a valid T_unit choice. Choose from ["K", "C"].'
        
    return df, T_label
    

def plot_phase_fractions(phase_frac_tbl, ax=None, T_unit='C'):
    mineral_frac_tbl = phase_frac_tbl.drop(columns=['Liquid','Water'])
    
    mineral_frac_tbl, T_label = _convert_temp_index(mineral_frac_tbl, T_unit)
        
    fTOL = 1e-4
    cols = mineral_frac_tbl.max(axis=0)>fTOL
    
    abundant_phs_tbl = mineral_frac_tbl.loc[::-1, cols].astype('float')

    abundant_phs_tbl.index = np.round(abundant_phs_tbl.index).astype('int')
    crystallization_tbl = pd.DataFrame(columns=abundant_phs_tbl.columns, 
                                       index=range(abundant_phs_tbl.index.min(),
                                                   abundant_phs_tbl.index.max()+1),
                                       dtype='float')
    
    crystallization_tbl.update(abundant_phs_tbl)
    crystallization_tbl = crystallization_tbl.interpolate(method='index')

    crystallization_tbl.plot.bar(stacked=True, ax=ax, width=1)

    if ax is None:
        ax = plt.gca()
        
    ax.set_xlabel(T_label)
    ax.set_ylabel('Mass Fraction')
    ax.set_xticks([])
    
def plot_magma_evolution(history, T_lims = None, T_unit='C'):
    phase_frac_tbl = history.phase_frac_table
    
    liq_comp = history.liquid_comp_table
    liq_comp, T_label = _convert_temp_index(liq_comp, T_unit)

    fig, ax = plt.subplots(nrows=3, sharex=False, squeeze=True, figsize=(5,10) )

    iax = ax[0]
    plot_phase_fractions(phase_frac_tbl, ax=iax, T_unit=T_unit)
    iax.set_xticklabels([])
    iax.set_xlabel('')

    iax=ax[1]
    liq_comp.plot(y=['MgO','FeO','Fe2O3','Al2O3','K2O','Na2O','H2O'], ax=iax).legend(loc='upper left')
    iax.set_ylabel('Magma Comp [wt%]')
    iax.set_xticklabels([])


    iax=ax[2]
    liq_comp.plot(y='SiO2', ax=iax, legend=True)
    iax.set_xlabel(T_label)
    iax.set_ylabel('Magma Comp [wt%]')
    
    if T_lims is not None:
        ax[1].set_xlim(T_lims)
        ax[2].set_xlim(T_lims)
    
    data_lims = [liq_comp.index.min(), liq_comp.index.max()]
        
    axT_lims = np.round(ax[2].get_xlim())
    
    _adjust_bar_plot_limits(data_lims, axT_lims, ax[0])
    
def _adjust_bar_plot_limits(data_lims, ax_lims, ax_bar:plt.axis) -> None:
        
    bar_plot_lims = ax_bar.get_xlim()

    def xpos_map(Tvalue):
        slope = (bar_plot_lims[1]-bar_plot_lims[0])/(data_lims[1]-data_lims[0])
        return bar_plot_lims[0] + (Tvalue-data_lims[0])*slope
    
    expand_xlims = [0,0]
    expand_xlims[0] = xpos_map(ax_lims[0])
    expand_xlims[1] = xpos_map(ax_lims[1])
    ax_bar.set_xlim(expand_xlims)
    

In [ ]:
comp={
    'BSE': {
        'SiO2': 45.97,
        'TiO2':  0.18,
        'Al2O3': 4.77,
        'Fe2O3': 1e-7,
        'FeO':   8.24,
        'MnO':   0.0,
        'MgO':  36.66,
        'CaO':   3.78,
        'Na2O':  0.35,
        'K2O':   0.04,},
    'BSE_MS95': {
        'SiO2': 45.5,
        'TiO2':  0.00001,
        'Al2O3': 4.50,
        'FeO':   8.15,
        'MnO':   0.0,
        'MgO':  38.3,
        'CaO':   3.58,
        'Na2O':  0.00001,
        'K2O':   0.0,},
}
# T0 = 1600.00+273+50
# P = 40e3
P = 24e3
T0 = 2200.0
Tfinal = 1600 + 273-100

BSE = comp['BSE_MS95']
dNNO = -5
# BSE = comp['BSE']

model_name = 'pMELTS'

In [ ]:

def adjust_init_redox(T0:float, P0:float, comp:dict, del_fO2:float, O2_buffer:str, model_name:str):
    sys_O2 = magmaforge.System(comp=comp, T0=T0, P=P0, del_fO2=del_fO2, O2_buffer=O2_buffer, 
                               model_name=model_name)
    # S0 = sys_O2.total_entropy
    # S0 = S0 * sys_O2.state.liquid.comp.sum()/sys_O2.state.properties.mass_tot

    comp_adj = sys_O2.state.liquid.comp
    return comp_adj


In [ ]:
comp_IW0 = adjust_init_redox(T0, P, BSE, dNNO, 'NNO', model_name)

In [ ]:
def get_liquidus_temp(T0, P, comp, model_name):
    
    def _step_down_to_liquidus(T0, Tstep, P=P, comp=comp, model_name=model_name):
        sys_T = magmaforge.System(comp=comp, T0=T0, P=P,
                                melt_frac_cutoff=.99, model_name=model_name)
        sys_T.crystallize(method='equil', Tstep=Tstep)

        T_liquidus = sys_T.history.get_temps()[sys_T.history.get_melt_frac()==1][-1]
        return T_liquidus
    
    T_liquidus = _step_down_to_liquidus(T0, 10)
    T_liquidus = _step_down_to_liquidus(T_liquidus, 1)
    
    return T_liquidus



In [ ]:
Tliq = get_liquidus_temp(T0, P, comp_IW0, model_name)

In [ ]:
Tliq

In [ ]:
comp_IW = adjust_init_redox(Tliq, P, BSE, dNNO, 'NNO', model_name)

In [ ]:
comp_IW-comp_IW0

In [ ]:
comp_IW-comp_IW0*comp_IW['SiO2']/comp_IW0['SiO2']

In [ ]:
T0 = Tliq

In [ ]:
sys_T = magmaforge.System(comp=comp_IW, T0=T0, P=P,
                          melt_frac_cutoff=.15, Tfinal=Tfinal, model_name=model_name)

S0 = sys_T.total_entropy
print(S0)

sys_S = magmaforge.System(comp=comp_IW, T0=T0, P=P, 
                          min_potential='H', S0=S0, model_name=model_name)

S0 = sys_S.total_entropy
print(S0)


In [ ]:

print(sys_T.state.conditions)
print(sys_T.state.liquid.comp['Fe2O3'])

print(sys_S.state.conditions)
print(sys_S.state.liquid.comp['Fe2O3'])

In [ ]:
sys_T.crystallize(method='equil', Tstep=5)

In [ ]:
plot_magma_evolution(sys_T.history)

In [ ]:
plt.figure()
plt.plot(sys_T.history.get_temps(), sys_T.history.get_melt_frac(),'.-')
plt.figure()
plt.plot(sys_T.history.get_temps(), sys_T.history.get_total_entropy(),'.-')

In [ ]:
T0

In [ ]:
while(sys_S.mass_fraction > 0.15):
    sys_S.cool(dS=1)
    print(sys_S.T)
    if sys_S.T < Tfinal:
        break

In [ ]:
plt.figure()
plt.plot(sys_S.history.get_temps()-273, sys_S.history.get_melt_frac(),'.-', label='S')
plt.plot(sys_T.history.get_temps()-273, sys_T.history.get_melt_frac(),'--', label='T')
plt.xlabel('T')
plt.ylabel('melt frac')
plt.legend()

plt.figure()
plt.plot(sys_S.history.get_temps()-273, sys_S.history.get_total_entropy(),'.-', label='S')
plt.plot(sys_T.history.get_temps()-273, sys_T.history.get_total_entropy(),'--', label='T')
plt.xlabel('T')
plt.ylabel('Stot')

In [ ]:
plot_magma_evolution(sys_T.history, T_lims=[1475, 1700])

In [ ]:
plot_magma_evolution(sys_S.history, T_lims=[1475, 1700])

In [ ]:
moon_press = pd.read_csv('data/Moon_press.csv',header=0)
moon_press

In [ ]:
def transform_mass(m):
    return m**(2/3)

def apply_constrained_poly_endpoints(poly_coef:np.array, Pmax:float):
    xends = np.array([0, 1])
    yends = np.polyval(poly_coef, [0,1])-(1-xends)*Pmax

    poly_coef = poly_coef.copy()
    poly_coef[-1] -= yends[0]
    poly_coef[-2] -= yends[1]-yends[0]
    return poly_coef

def fit_press_poly(mass_frac:np.ndarray, P:np.ndarray, deg:int=4):
    mval = transform_mass(mass_frac)

    poly_coef0 = np.polyfit(mval, P, deg)
    Pmax = P.max()
    poly_coef = apply_constrained_poly_endpoints(poly_coef0, Pmax)
    return poly_coef

def eval_press(mass_frac:np.ndarray, poly_coef:np.ndarray):
    if type(mass_frac) is not np.ndarray:
        mass_frac = np.array(mass_frac)
        
    mval = transform_mass(mass_frac)
    return np.polyval(poly_coef, mval)
    


m_frac = np.linspace(0, 1,101)

poly_coef = fit_press_poly(moon_press['mass_frac'], moon_press['P'])


mval = transform_mass(moon_press['mass_frac'])
mmod = transform_mass(m_frac)

# poly_coef = np.polyfit(mval, moon_press['P'],4)
Pmax = moon_press['P'].max()
# poly_coef = apply_constrained_poly_endpoints(poly_coef, Pmax)




plt.figure()
plt.plot(mval, moon_press['P']-(1-mval)*Pmax, '.-')
# plt.plot(mmod, np.polyval(poly_coef, mmod)-(1-mmod)*Pmax-correction(mmod), 'r--')

plt.plot(mmod, np.polyval(poly_coef, mmod)-(1-mmod)*Pmax, 'r--')



In [ ]:
eval_press([0,1], poly_coef)

In [ ]:
plt.figure()
plt.plot(moon_press['mass_frac'], moon_press['P'], 'o')
plt.plot(m_frac, eval_press(m_frac, poly_coef), 'r--')
plt.plot(m_frac, (1-m_frac)*Pmax, ':', color=[.5,.5,.5])

mass_samples = 1-.9**np.arange(15)
plt.plot(mass_samples, eval_press(mass_samples, poly_coef), 'mx')

In [ ]:
poly_coef = np.polyfit(moon_press['mass_frac']**.5, moon_press['P'],2)

m_frac = np.linspace(0, 1,101)
plt.figure()
plt.plot(moon_press['mass_frac']**.5, moon_press['P'], '.-')
plt.plot(m_frac**.5, np.polyval(poly_coef, m_frac**.5), 'r--')